### Thanks to : https://www.kaggle.com/code/digantabhattacharya/neurips25-rdkit-xgb-additionaldata-starter-i
### Thanks to : https://www.kaggle.com/code/richolson/smiles-rdkit-lgbm-ftw for the additional datasets.
Addtional dataset for target 'Tg' and 'Tc'： Tg🚫: Confirmed as unusable.

### Baseline XGBoost with RDKit Feature Extraction : https://www.kaggle.com/code/digantabhattacharya/neurips25-opp-with-rdkit-xgb-starter-i/
### Baseline LGBM with RDKit Feature Extraction : https://www.kaggle.com/code/digantabhattacharya/neurips25-opp-with-rdkit-lgbm-starter-ii/ [In Progress]

The main changes are as follows:

Comprehensive Feature Expansion:

Changes: The original restriction of truncating or padding feature vectors to 200 dimensions has been completely removed. The new scheme calculates all available approximately 200 molecular descriptors in the RDKit library and expands the Morgan fingerprint to a full 2048 bits.
Purpose: Aims to provide the most comprehensive chemical information to the model, avoid the loss of potential effective signals due to artificial truncation, and theoretically allow the model to capture more complex structure-activity relationships.
Introducing a cross-validation mechanism (Robust Cross-Validation):

Changes: Replace the single train_test_split validation method in the original text with a more robust 5-fold cross-validation (K-Fold).
Purpose: Whether using Optuna for hyperparameter optimization or in the final training prediction stage, cross-validation can provide a more reliable model performance evaluation, avoid the randomness of the results, and obtain more stable submission results by averaging the predictions of 5 models.
SMILES Canonicalization:

Changes: Added a preprocessing step to convert all SMILES strings into a unique "standard" representation.

Purpose: Ensure that the same molecular structure, regardless of its SMILES writing, is considered the same input to the model, eliminating data redundancy and inconsistency.

In [1]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl


# ----------------------------------------------------------------------------
# 1. Import library and environment settings
# ----------------------------------------------------------------------------


In [2]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
import logging
import os
import joblib
from tqdm.notebook import tqdm

# RDKit related libraries
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator

# Machine learning related libraries
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import optuna

# ----------------------------------------------------------------------------
# 2. Log and warning configuration
# ----------------------------------------------------------------------------


In [3]:
# Suppress all warning messages
warnings.filterwarnings("ignore")

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(), # output to console
        logging.FileHandler('polymer_prediction_optimized.log') # 輸出到文件
    ]
)
logger = logging.getLogger(__name__)


# ----------------------------------------------------------------------------
# 3. Helper function
# ----------------------------------------------------------------------------


In [4]:

def check_gpu_availability() -> Dict:
    """Checks whether the GPU is available and returns appropriate XGBoost parameters."""
    try:
        # Try to initialize a simple model using GPU parameters
        xgb.XGBRegressor(tree_method='gpu_hist').get_params()
        logger.info("GPU detected! GPU acceleration will be enabled for XGBoost.")
        return {
            'tree_method': 'gpu_hist',
            'gpu_id': 0,
            'predictor': 'gpu_predictor'
        }
    except Exception as e:
        logger.warning(f"GPU is not available: {str(e)}.Will fall back to the CPU for computation.")
        return {
            'tree_method': 'hist',
            'predictor': 'cpu_predictor'
        }

def canonicalize_smiles(smiles: str) -> str:
    """Convert a SMILES string to its canonical representation, guaranteed to be unique."""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return Chem.MolToSmiles(mol, canonical=True)
    except Exception:
        # If SMILES is invalid, returns None
        return None
    return None

# ----------------------------------------------------------------------------
# 4. Data loading and preprocessing
# ----------------------------------------------------------------------------
Addtional dataset for target 'Tg' and 'Tc'： Tg🚫: Confirmed as unusable.

In [5]:

def load_data() -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load competition data and additional datasets, and perform normalization and deduplication."""
    # logger.info("Starting to load contest data...")
    # comp_train_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
    # test_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
    # logger.info(f"{len(comp_train_df)} competition training samples and {len(test_df)} test samples loaded.")

    # logger.info("Starting loading additional datasets...")
    # extra_tg_df = pd.read_csv("/kaggle/input/smiles-tg/Tg_SMILES_class_pid_polyinfo_median.csv")
    # extra_tc_df = pd.read_csv("/kaggle/input/tc-smiles/Tc_SMILES.csv")
    # logger.info(f"{len(extra_tg_df)} extra Tg samples and {len(extra_tc_df)} extra Tc samples loaded.")

    # # Extract only the columns you need to simplify merging
    # comp_train_df_slim = comp_train_df[['SMILES', 'Tg', 'FFV', 'Tc', 'Density', 'Rg']]
    # extra_tg_clean = extra_tg_df[['SMILES', 'Tg']].copy()
    # extra_tc_clean = extra_tc_df[['SMILES', 'TC_mean']].rename(columns={'TC_mean': 'Tc'}).copy()

    # # Merge all training data
    # train_df = pd.concat([comp_train_df_slim, extra_tg_clean, extra_tc_clean], ignore_index=True)
    
    # logger.info(f"The total number of training samples after merging is {len(train_df)}。")
    # logger.info("Beginning the process of standardizing SMILES...")
    
    # # Perform SMILES normalization on the training and test sets
    # train_df['SMILES'] = train_df['SMILES'].apply(canonicalize_smiles)
    # test_df['SMILES'] = test_df['SMILES'].apply(canonicalize_smiles)
    
    # # Remove rows with invalid SMILES and remove duplicates based on the normalized SMILES
    # train_df.dropna(subset=['SMILES'], inplace=True)
    # train_df.drop_duplicates(subset=['SMILES'], keep='first', inplace=True)
    
    # logger.info(f"After standardization and duplicate removal, the number of remaining valid training samples is {len(train_df)}.")
    
    # return train_df.reset_index(drop=True), test_df
    
    logger.info("Starting to load contest data...")
    comp_train_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
    test_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
    
    # --- Modification section starts ---
    logger.info("Starting loading additional datasets...")
    # extra_tg_df = pd.read_csv("/kaggle/input/smiles-tg/Tg_SMILES_class_pid_polyinfo_median.csv") # 已移除
    extra_tc_df = pd.read_csv("/kaggle/input/tc-smiles/Tc_SMILES.csv")
    logger.info(f"{len(extra_tc_df)} extra Tc samples loaded.")

    comp_train_df_slim = comp_train_df[['SMILES', 'Tg', 'FFV', 'Tc', 'Density', 'Rg']]
    extra_tc_clean = extra_tc_df[['SMILES', 'TC_mean']].rename(columns={'TC_mean': 'Tc'}).copy()
    
    # Merge only the race data and the extra Tc data
    train_df = pd.concat([comp_train_df_slim, extra_tc_clean], ignore_index=True)
    # --- End of modification section ---
    
    logger.info(f"The total number of training samples after merging is {len(train_df)}.")
    logger.info("Standardize SMILES and remove duplicates...")
    train_df['SMILES'] = train_df['SMILES'].apply(canonicalize_smiles)
    test_df['SMILES'] = test_df['SMILES'].apply(canonicalize_smiles)
    train_df.dropna(subset=['SMILES'], inplace=True)
    train_df.drop_duplicates(subset=['SMILES'], keep='first', inplace=True)
    
    logger.info(f"After standardization and deduplication, the number of remaining valid training samples is {len(train_df)}.")
    return train_df.reset_index(drop=True), test_df


# ----------------------------------------------------------------------------
# 5. Core model class:PolymerPredictor
# ----------------------------------------------------------------------------


In [6]:

class PolymerPredictor:
    """
    A complete machine learning pipeline for predicting polymer properties.
    """
    def __init__(self, random_state=42):
        logger.info("Initialize PolymerPredictor...")
        self.random_state = random_state
        self.xgb_params = check_gpu_availability()
        
        # --- Corrected parts ---
        self.descriptor_calculators = [desc[1] for desc in Descriptors._descList]
        
        # Store the fingerprint size as an attribute
        self.fp_size = 2048
        self.morgan_gen = GetMorganGenerator(radius=3, fpSize=self.fp_size) 
        
        # Dynamically determine the total length of the feature vector
        num_descriptors = len(self.descriptor_calculators)
        # Directly use the fingerprint size attribute we set
        num_fp_bits = self.fp_size
        self.feature_length = num_descriptors + num_fp_bits
        logger.info(f"The dimension of the feature vector has been determined to be: {self.feature_length} ({num_descriptors} descriptors + {num_fp_bits} fingerprints)")
        # --- End of revision ---
        
        self.models = {}  # Store the model for each attribute
        self.scalers = {} # Stores the scaler for each attribute

    def _extract_features(self, smiles_list: List[str]) -> np.ndarray:
        """Efficiently extract rich chemical features from a list of SMILES strings."""
        logger.info(f"Start extracting features from {len(smiles_list)} SMILES..​​.")
        
        features = []
        for smiles in tqdm(smiles_list, desc="Feature extraction"):
            try:
                mol = Chem.MolFromSmiles(smiles)
                if mol is None:
                    features.append(np.zeros(self.feature_length))
                    continue
                
                # Calculate all RDKit descriptors
                descriptors = [func(mol) for func in self.descriptor_calculators]
                
                # Calculating Morgan fingerprint
                fp = self.morgan_gen.GetFingerprint(mol)
                fp_bits = [int(x) for x in fp.ToBitString()]
                
                feature_vector = descriptors + fp_bits
                features.append(feature_vector)
            except Exception as e:
                logger.warning(f"Error processing SMILES '{smiles}': {e}")
                features.append(np.zeros(self.feature_length))
        
        # Convert the result to a numpy array and handle the inf or nan values ​​that may be generated in the calculation
        feature_array = np.array(features, dtype=np.float32)
        feature_array = np.nan_to_num(feature_array, nan=0.0, posinf=0.0, neginf=0.0)
        
        return feature_array

    def optimize_hyperparameters(self, X: np.ndarray, y: pd.Series, n_trials: int = 50) -> Dict:
        """Use Optuna and cross-validation to optimize XGBoost hyperparameters for a single attribute to get more robust results."""
        logger.info(f"Starting {n_trials} Optuna hyperparameter optimization for target attribute...")

        def objective(trial: optuna.Trial) -> float:
            # Define a wider hyperparameter search space for XGBoost
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 1000, 8000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'max_depth': trial.suggest_int('max_depth', 5, 20),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 2.0, log=True),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                **self.xgb_params,
                'objective': 'reg:squarederror', 
                'random_state': self.random_state
            }
            
            # Use 5-fold cross validation to evaluate the performance of each set of hyperparameters
            kf = KFold(n_splits=5, shuffle=True, random_state=self.random_state)
            mae_scores = []
            
            for train_idx, valid_idx in kf.split(X):
                X_train, X_valid = X[train_idx], X[valid_idx]
                y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
                
                model = xgb.XGBRegressor(**params)
                model.fit(X_train, y_train, 
                          eval_set=[(X_valid, y_valid)], 
                          early_stopping_rounds=50, 
                          verbose=False)
                
                preds = model.predict(X_valid)
                mae_scores.append(mean_absolute_error(y_valid, preds))
            
            return np.mean(mae_scores)

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        
        logger.info(f"Optimization completed. Average cross-validation MAE: {study.best_value:.5f}")
        logger.info(f"The best parameters found: {study.best_params}")
        return study.best_params

    def train_and_predict_with_cv(self, X_train: np.ndarray, y_train: pd.Series, X_test: np.ndarray, best_params: Dict) -> Tuple[np.ndarray, list, StandardScaler]:
        """The model was trained using 5-fold cross validation and ensemble predictions were performed on the test set."""
        test_predictions_per_fold = []
        trained_models = []
        
        final_params = best_params.copy()
        final_params.update(self.xgb_params)
        final_params['objective'] = 'reg:squarederror'
        final_params['random_state'] = self.random_state
        final_params['early_stopping_rounds'] = 100
        
        kf = KFold(n_splits=5, shuffle=True, random_state=self.random_state)

        # standardized features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        for fold, (train_idx, _) in enumerate(kf.split(X_train_scaled, y_train)):
            logger.info(f"--- Start training the {fold+1}/5th fold model ---")
            
            X_fold_train, y_fold_train = X_train_scaled[train_idx], y_train.iloc[train_idx]
            
            # Train a model for a single fold using the best parameters
            model = xgb.XGBRegressor(**final_params)
            model.fit(X_fold_train, y_fold_train, eval_set=[(X_fold_train, y_fold_train)], verbose=1000)
            
            # Make predictions on the test set and store them
            preds = model.predict(X_test_scaled)
            test_predictions_per_fold.append(preds)
            trained_models.append(model)
        
        # Take the average of the prediction results of the five models as the final integrated prediction
        final_predictions = np.mean(test_predictions_per_fold, axis=0)
        
        return final_predictions, trained_models, scaler

# ----------------------------------------------------------------------------
# 6.Main execution process
# ----------------------------------------------------------------------------


In [7]:
if __name__ == "__main__":
    # Create folders to save models and results
    os.makedirs('/kaggle/working/models', exist_ok=True)
    

    train_df, test_df = load_data()

    predictor = PolymerPredictor()
    
    X_test_features = predictor._extract_features(test_df['SMILES'].tolist())
    
    final_submission_predictions = {}
    
    TARGET_COLUMNS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
    
    for prop in TARGET_COLUMNS:
        logger.info(f"\n{'='*25} Processing properties: {prop} {'='*25}")
        
        prop_train_df = train_df.dropna(subset=[prop]).reset_index(drop=True)
        if prop_train_df.empty:
            logger.warning(f"Property {prop} has no valid training data and will be skipped.")
            continue
            
        X_prop_train = predictor._extract_features(prop_train_df['SMILES'].tolist())
        y_prop_train = prop_train_df[prop]
        logger.info(f"Number of valid training samples for property {prop}: {len(prop_train_df)}")

        best_params_for_prop = predictor.optimize_hyperparameters(X_prop_train, y_prop_train, n_trials=75)
        
        logger.info(f"Use the best parameters found to train {prop} with 5-fold cross validation...")
        prop_predictions, prop_models, prop_scaler = predictor.train_and_predict_with_cv(
            X_prop_train, y_prop_train, X_test_features, best_params_for_prop
        )
        
        final_submission_predictions[prop] = prop_predictions
        
        logger.info(f"Save the model and scaler for property {prop}...")
        joblib.dump(prop_models, f'/kaggle/working/models/{prop}_models.joblib')
        joblib.dump(prop_scaler, f'/kaggle/working/models/{prop}_scaler.joblib')

    logger.info("All properties processed, creating commit file...")
    submission_df = pd.DataFrame({'id': test_df['id']})
    for prop in TARGET_COLUMNS:
        if prop in final_submission_predictions:
            submission_df[prop] = final_submission_predictions[prop]
        else:
            submission_df[prop] = 0 
            
    submission_df.to_csv('/kaggle/working/submission.csv', index=False)
    
    logger.info("The submission file submission.csv has been generated successfully!")
    logger.info("The process is complete!")


Feature extraction:   0%|          | 0/3 [00:00<?, ?it/s]

Feature extraction:   0%|          | 0/511 [00:00<?, ?it/s]

[I 2025-06-26 13:55:02,030] A new study created in memory with name: no-name-e5ee0104-a94e-4e49-8234-d2d93f6f25b8


  0%|          | 0/75 [00:00<?, ?it/s]

[I 2025-06-26 13:55:08,328] Trial 0 finished with value: 49.92562074956827 and parameters: {'n_estimators': 7508, 'learning_rate': 0.03667545021437271, 'max_depth': 8, 'subsample': 0.8144254556745026, 'colsample_bytree': 0.8262232017220603, 'gamma': 0.012350553070661748, 'reg_alpha': 0.027351602494057853, 'reg_lambda': 4.7108389550885785e-07, 'min_child_weight': 10}. Best is trial 0 with value: 49.92562074956827.
[I 2025-06-26 13:55:13,020] Trial 1 finished with value: 51.12414231361703 and parameters: {'n_estimators': 5090, 'learning_rate': 0.055405233156165204, 'max_depth': 8, 'subsample': 0.9324369112657941, 'colsample_bytree': 0.729106126450912, 'gamma': 0.021841995523532224, 'reg_alpha': 0.2874764847191478, 'reg_lambda': 9.9479812781825e-05, 'min_child_weight': 4}. Best is trial 0 with value: 49.92562074956827.
[I 2025-06-26 13:55:24,570] Trial 2 finished with value: 50.0198137952808 and parameters: {'n_estimators': 5771, 'learning_rate': 0.017576236699690388, 'max_depth': 15, 'su

...

[I 2025-06-26 17:38:38,908] Trial 0 finished with value: 0.06737470860420718 and parameters: {'n_estimators': 7555, 'learning_rate': 0.07103984706187096, 'max_depth': 11, 'subsample': 0.7619692962262472, 'colsample_bytree': 0.6594454650065077, 'gamma': 0.4478217449215358, 'reg_alpha': 0.5831124765333068, 'reg_lambda': 1.669875436232546e-08, 'min_child_weight': 7}. Best is trial 0 with value: 0.06737470860420718.
[I 2025-06-26 17:38:43,273] Trial 1 finished with value: 0.03483816873224077 and parameters: {'n_estimators': 2826, 'learning_rate': 0.09102486950200211, 'max_depth': 13, 'subsample': 0.678932583674133, 'colsample_bytree': 0.8075830979590659, 'gamma': 0.0004153986418272754, 'reg_alpha': 0.05127422903580848, 'reg_lambda': 0.08467074142389902, 'min_child_weight': 3}. Best is trial 1 with value: 0.03483816873224077.
[I 2025-06-26 17:38:50,550] Trial 2 finished with value: 0.0324986032056695 and parameters: {'n_estimators': 1993, 'learning_rate': 0.04242626279942006, 'max_depth': 1

Feature extraction:   0%|          | 0/614 [00:00<?, ?it/s]

[I 2025-06-26 18:04:07,584] A new study created in memory with name: no-name-d46a935e-3b39-4aef-b57e-7d42fe4df573


  0%|          | 0/75 [00:00<?, ?it/s]

[I 2025-06-26 18:04:22,835] Trial 0 finished with value: 1.7310988600561898 and parameters: {'n_estimators': 4064, 'learning_rate': 0.021443111784990615, 'max_depth': 10, 'subsample': 0.9613751639107122, 'colsample_bytree': 0.9421186450494179, 'gamma': 8.019365259676585e-05, 'reg_alpha': 1.296676578349975e-06, 'reg_lambda': 9.420006177748326e-05, 'min_child_weight': 3}. Best is trial 0 with value: 1.7310988600561898.
[I 2025-06-26 18:05:06,643] Trial 1 finished with value: 1.591424331078511 and parameters: {'n_estimators': 3739, 'learning_rate': 0.018276870428863296, 'max_depth': 20, 'subsample': 0.6767747921922546, 'colsample_bytree': 0.6289319997301253, 'gamma': 5.188659722155204e-07, 'reg_alpha': 0.0011695773685154003, 'reg_lambda': 0.0749581614212616, 'min_child_weight': 1}. Best is trial 1 with value: 1.591424331078511.
[I 2025-06-26 18:05:11,890] Trial 2 finished with value: 1.6330740872729508 and parameters: {'n_estimators': 2334, 'learning_rate': 0.057377923143619204, 'max_dept